In [1]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode
import numpy as np
from shapely.geometry import Point, Polygon
import geopandas as gpd


import matplotlib.pyplot as plt

In [30]:
crime2023 = pd.read_excel('/Users/pablouriarte/Downloads/Municipal-Delitos-2015-2024_mar2024/2023.xlsx')


In [47]:
mx_mun = gpd.read_file('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/2. arcgis(general)/maps_boundaries/Municipalities/mapa_mexico_mun/Division_Municipal_Mexico_2010.shp').set_index('CLAVE').to_crs(epsg=4485)

In [55]:
# Crear la columna 'Cve. Municipio' concatenando 'CVE_EDO' y 'CVE_MUNI'
mx_mun['Cve. Municipio'] = mx_mun['CVE_EDO'].astype(str).str.zfill(2) + mx_mun['CVE_MUNI'].astype(str).str.zfill(3)

# Convertir la nueva columna a tipo entero para eliminar cualquier cero inicial no deseado
mx_mun['Cve. Municipio'] = mx_mun['Cve. Municipio'].astype(int)

# Convertir de nuevo a string si es necesario mantener la columna como texto
mx_mun['Cve. Municipio'] = mx_mun['Cve. Municipio'].astype(str)


In [54]:
mx_mun.tail(60)

,NOM_MUN,NOMEDO,CVE_EDO,CVE_MUNI,Area,geometry,Cve. Municipio
CLAVE,,,,,,,
20211,San Juan Ozolotepec,Oaxaca,20,211,277.538082,"POLYGON ((2089249.153 1849744.429, 2089767.136...",20211
20307,San Pedro Huamelula,Oaxaca,20,307,808.290378,"POLYGON ((2150674.419 1842868.502, 2151154.228...",20307
20254,San Mateo Rio Hondo,Oaxaca,20,254,227.128757,"POLYGON ((2065859.096 1845206.987, 2067042.099...",20254
20424,Santa Maria Ozolotepec,Oaxaca,20,424,137.541697,"POLYGON ((2077754.797 1844349.049, 2077845.882...",20424
20433,Santa Maria Temaxcaltepec,Oaxaca,20,433,31.405336,"POLYGON ((1987575.359 1839898.529, 1989549.557...",20433
07034,Frontera Comalapa,Chiapas,07,034,774.133809,"POLYGON ((2545931.244 1852417.810, 2546159.872...",7034
07030,Chicomuselo,Chiapas,07,030,968.935328,"POLYGON ((2527045.048 1853424.815, 2528353.030...",7030
20526,Santos Reyes Nopala,Oaxaca,20,526,225.744593,"POLYGON ((1982227.223 1838261.344, 1983101.909...",20526
20366,Santa Catarina Loxicha,Oaxaca,20,366,128.083945,"POLYGON ((2035796.089 1836449.269, 2036674.228...",20366


In [32]:
import pandas as pd

# Assuming 'crime2023' is your DataFrame
# Display unique values in the 'Tipo de delito' column
unique_types = crime2023['Tipo de delito'].unique()
print("Unique values in 'Tipo de delito':")
print(unique_types)

# Display unique values in the 'Bien jurídico afectado' column
unique_subtypes = crime2023['Bien jurídico afectado'].unique()
print("\nUnique values in 'Bien jurídico afectado':")
print(unique_subtypes)


Unique values in 'Tipo de delito':
['Homicidio' 'Lesiones' 'Feminicidio' 'Aborto'
 'Otros delitos que atentan contra la vida y la integridad corporal'
 'Secuestro' 'Tráfico de menores' 'Rapto'
 'Otros delitos que atentan contra la libertad personal' 'Abuso sexual'
 'Acoso sexual' 'Hostigamiento sexual' 'Violación simple'
 'Violación equiparada' 'Incesto'
 'Otros delitos que atentan contra la libertad y la seguridad sexual'
 'Robo' 'Fraude' 'Abuso de confianza' 'Extorsión' 'Daño a la propiedad'
 'Despojo' 'Otros delitos contra el patrimonio' 'Violencia familiar'
 'Violencia de género en todas sus modalidades distinta a la violencia familiar'
 'Incumplimiento de obligaciones de asistencia familiar'
 'Otros delitos contra la familia' 'Corrupción de menores'
 'Trata de personas' 'Otros delitos contra la sociedad' 'Narcomenudeo'
 'Amenazas' 'Allanamiento de morada' 'Evasión de presos' 'Falsedad'
 'Falsificación' 'Contra el medio ambiente'
 'Delitos cometidos por servidores públicos' 'Electo

In [33]:
# Filtramos primero por las categorías principales de 'Bien jurídico afectado'
categorias_deseadas = [
    "La libertad y la seguridad sexual",
    "La sociedad",
    "La vida y la Integridad corporal",
    "Libertad personal",
    "Otros bienes jurídicos afectados (del fuero común)"
]

filtered_crime2023 = crime2023[crime2023['Bien jurídico afectado'].isin(categorias_deseadas)]

# Ahora definimos las subcategorías específicas para filtrar en 'Tipo de delito'
# según corresponda a la categoría principal
subcategorias_filtro = {
    "La vida y la Integridad corporal": ["Feminicidio"],
    "Libertad personal": ["Tráfico de menores"],
    "Otros bienes jurídicos afectados (del fuero común)": ["Narcomenudeo", "Delitos cometidos por servidores públicos"]
}

# Aplicar filtro adicional basado en subcategorías específicas
def filtro_adicional(row):
    if row['Bien jurídico afectado'] in subcategorias_filtro:
        # Si hay subcategorías específicas, filtrar por ellas
        return row['Tipo de delito'] in subcategorias_filtro[row['Bien jurídico afectado']]
    else:
        # Si no hay subcategorías específicas, mantener la fila
        return True

# Aplicamos el filtro adicional
filtered_crime2023 = filtered_crime2023[filtered_crime2023.apply(filtro_adicional, axis=1)]



In [34]:
# Eliminar la columna "Modalidad"
filtered_crime2023 = filtered_crime2023.drop('Modalidad', axis=1)

# Lista de nombres de las columnas de los meses
meses = [
    "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", 
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

# Sumar las columnas de los meses para cada fila
filtered_crime2023['Total delitos'] = filtered_crime2023[meses].sum(axis=1)

# Eliminar las columnas individuales de los meses si ya no se necesitan
filtered_crime2023 = filtered_crime2023.drop(columns=meses)

In [40]:
def etiquetar(row):
    if row['Bien jurídico afectado'] == 'La vida y la Integridad corporal' and row['Tipo de delito'] == 'Feminicidio':
        return 'Violencia Mujeres, Familias y Niños'
    elif row['Bien jurídico afectado'] == 'Libertad personal' and row['Tipo de delito'] == 'Tráfico de menores':
        return 'Violencia Mujeres, Familias y Niños'
    elif row['Bien jurídico afectado'] == 'La libertad y la seguridad sexual':
        return 'Violencia Mujeres, Familias y Niños'
    elif row['Bien jurídico afectado'] == 'La sociedad':
        return 'Violencia Mujeres, Familias y Niños'
    
    elif row['Bien jurídico afectado'] == 'Otros bienes jurídicos afectados (del fuero común)' and row['Tipo de delito'] == 'Narcomenudeo':
        return 'Crimen Organizado'
    
    elif row['Bien jurídico afectado'] == 'Otros bienes jurídicos afectados (del fuero común)' and row['Tipo de delito'] == 'Delitos cometidos por servidores públicos':
        return 'Corrupción Institucional'
    else:
        return 'Otra categoría'  # o puedes elegir un valor predeterminado más específico

# Aplicar la función para crear la nueva columna de etiquetas
filtered_crime2023['Etiqueta'] = filtered_crime2023.apply(etiquetar, axis=1)


In [44]:
# Eliminar las columnas que no se necesitan
columns_to_drop = ['Bien jurídico afectado', 'Tipo de delito', 'Subtipo de delito']
df_reduced = filtered_crime2023.drop(columns=columns_to_drop)

# Agrupar por 'Cve. Municipio' y 'Etiqueta', y sumar los valores de 'Total delitos'
df_aggregated = df_reduced.groupby(['Cve. Municipio', 'Etiqueta']).agg({'Total delitos': 'sum'}).reset_index()


In [56]:
df_aggregated.head()

,Entidad,Municipio,Cve. Municipio,Etiqueta,Total delitos
0,Aguascalientes,Aguascalientes,1001,Corrupción Institucional,474
1,Aguascalientes,Aguascalientes,1001,Crimen Organizado,1462
2,Aguascalientes,Aguascalientes,1001,"Violencia Mujeres, Familias y Niños",1011
3,Aguascalientes,Asientos,1002,Corrupción Institucional,4
4,Aguascalientes,Asientos,1002,Crimen Organizado,108


In [63]:
# Seleccionar las columnas necesarias de 'mx_mun'
mx_mun_reduced = mx_mun[['Cve. Municipio', 'NOM_MUN', 'Area', 'geometry']]

# Convertir 'Cve. Municipio' en ambos DataFrames a tipo int para asegurar una comparación correcta
mx_mun_reduced['Cve. Municipio'] = mx_mun_reduced['Cve. Municipio'].astype(int)
df_aggregated['Cve. Municipio'] = df_aggregated['Cve. Municipio'].astype(int)

# Realizar el merge
df_final = pd.merge(df_aggregated, mx_mun_reduced, on='Cve. Municipio', how='left')


/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [64]:
df_final

,Entidad,Municipio,Cve. Municipio,Etiqueta,Total delitos,NOM_MUN,Area,geometry
0,Aguascalientes,Aguascalientes,1001,Corrupción Institucional,474,Aguascalientes,1168.762384,"POLYGON ((1416489.577 2467700.472, 1417908.226..."
1,Aguascalientes,Aguascalientes,1001,Crimen Organizado,1462,Aguascalientes,1168.762384,"POLYGON ((1416489.577 2467700.472, 1417908.226..."
2,Aguascalientes,Aguascalientes,1001,"Violencia Mujeres, Familias y Niños",1011,Aguascalientes,1168.762384,"POLYGON ((1416489.577 2467700.472, 1417908.226..."
3,Aguascalientes,Asientos,1002,Corrupción Institucional,4,Asientos,547.762077,"POLYGON ((1417043.958 2491681.240, 1417408.488..."
4,Aguascalientes,Asientos,1002,Crimen Organizado,108,Asientos,547.762077,"POLYGON ((1417043.958 2491681.240, 1417408.488..."
...,...,...,...,...,...,...,...,...
7561,Zacatecas,Villanueva,32055,Crimen Organizado,9,Villanueva,2158.696446,"POLYGON ((1345756.044 2531084.706, 1346503.579..."
7562,Zacatecas,Villanueva,32055,"Violencia Mujeres, Familias y Niños",15,Villanueva,2158.696446,"POLYGON ((1345756.044 2531084.706, 1346503.579..."
7563,Zacatecas,Zacatecas,32056,Corrupción Institucional,90,Zacatecas,444.547742,"POLYGON ((1339206.596 2548649.944, 1340025.280..."
7564,Zacatecas,Zacatecas,32056,Crimen Organizado,71,Zacatecas,444.547742,"POLYGON ((1339206.596 2548649.944, 1340025.280..."


In [65]:
# Guardar el DataFrame final en un archivo CSV
path = "/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/areas/riskzone.csv"
df_final.to_csv(path, index=False)


In [ ]:
import gzip
import shutil

# Ruta del archivo original
path_original = "/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/areas/riskzone.csv"
# Ruta del archivo comprimido
path_comprimido = "/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/areas/riskzone.csv.gz"

# Abrir el archivo original y el archivo comprimido
with open(path_original, 'rb') as f_in, gzip.open(path_comprimido, 'wb') as f_out:
    # Copiar el contenido del archivo original al archivo comprimido usando gzip
    shutil.copyfileobj(f_in, f_out)

print("Archivo comprimido con éxito.")
